<a href="https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For testing only

In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="black">Stable Diffusion 시작하는 섹션</font>
output_path = 'digiclau_lab'
#@markdown ## 🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩
#@markdown ## (선택사항) ngrok Authtoken을 이곳에 넣으면, ngrok GUI가 사용됩니다..
#@markdown ### 이곳을 공백으로 남겨두면, Gradio GUI가 기본으로 사용됩니다.
ngrok ='' #@param {type: "string"}

#@markdown ## 🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀
#@markdown ## 구글 드라이브에 연결할지를 결정합니다.
#@markdown ### 'Yes' 를 선택하면, 구글 드라이브에 'digiclau_lab'이라는 폴더가 생기고, 그 안에 세팅 파일과, 생성된 이미지들이 저장됩니다.
#@markdown ### 'No' 를 선택하면, 구글 드라이브는 필요없지만, 세팅설정과 생성된 이미지들은 저장되지 않습니다.
use_gdrive = 'yes' #@param ['yes', 'no']
class GDriveSaveMode:
    ON = 1
    OFF = 0
if use_gdrive == 'yes':
  is_drive_used = True
elif use_gdrive == 'no':
  is_drive_used = False
else:
  raise ValueError("use_gdrive value invalid.")

#@markdown ## 🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷
#@markdown ## 아래의 Checkpoint 모델중에, 사용할 모델을 선택하세요.
#@markdown ### 요청에 따라 모델을 추가하거나 변경할 수 있습니다.
#@markdown ### (처음 시작할때는 Stable Diffusion 1.5 모델이 기본으로 설치됩니다.)
#@markdown ### (사용할 모델만 선택하세요. 모델이 많을수록 시작하는데 시간이 더 걸립니다.)
Deliberate_v2 = False #@param{type: "boolean"}
DreamShaper_7 = False #@param{type: "boolean"}
GhostMix_v20 = False #@param{type: "boolean"}
RealisticVision_V30 = False #@param{type: "boolean"}
MajicMIXRealistic_v6 = False #@param{type: "boolean"}
AbyssOrangeMix3 = False #@param{type: "boolean"}
CounterfeitV30_v30 = False #@param{type: "boolean"}
ReVAnimated_v122 = False #@param{type: "boolean"}
Lyriel_v16 = False #@param{type: "boolean"}
NeverEndingDream_v122 = False #@param{type: "boolean"}

#@markdown ## 🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟
#@markdown ## 아래의 확장 프로그램(Extensions)중에, 사용할 모델을 선택하세요.
#@markdown ### 요청에 따라 모델을 추가하거나 변경할 수 있습니다.
#@markdown ### (사용할 모델만 선택하세요. 모델이 많을수록 시작하는데 시간이 더 걸립니다.)
#@markdown ### (예, ControlNet은 큰 파일들을 많이 사용합니다.)
ControlNet = False #@param{type: "boolean"}
Deforum = False #@param{type: "boolean"}
Regional_Prompter = False #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
Openpose_Editor = False #@param{type: "boolean"}
ADetailer = False #@param{type: "boolean"}

# 로그 삭제
#Clear_Log = True #@param{type:'boolean'}

# #@markdown ## ➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿
# #@markdown ## 위에 있는 Checkpoint 모델 외에 사용하고 싶은 모델의 다운로드 링크를 넣어주세요.
# #@markdown ### Hugging Face 와 civitAI만 가능합니다.
# #@markdown ### 각각의 링크 주소는 쉼표(,)로 구분해주세요.
# Model_URL = '' #@param {type: "string"}

# #@markdown ## ☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️
# #@markdown ## 위에 있는 확장 프로그램(Extensions) 외에 사용하고 싶은 모델의 URL을 넣어주세요.
# #@markdown ### GitHub만 가능합니다.
# #@markdown ### 각각의 URL 주소는 쉼표(,)로 구분해주세요.
# Extensions_URL = '' #@param {type: "string"}

# #@markdown ## 🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨
# #@markdown ## 'COMMANDLINE_ARGS=' 에 추가할 값이 있다면 이곳에 넣어주세요.
# Extra_args = '' #@param {type: "string"}


if is_drive_used:
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  # Colab에 설치
  root = '/content/'
  # 구글 드라이브에 필요한 폴터 생성
  !mkdir -p {output_path}/outputs
  !mkdir -p {output_path}/checkpoints
  !mkdir -p {output_path}/ESRGAN
  !mkdir -p {output_path}/hypernetworks
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path # Is it still needed?
  root = '/content/'

def print_message(msg):
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  # print('📍📍📍 ', msg, ' 📍📍📍')
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  pass

def clear():
    from IPython.display import clear_output
    return clear_output()

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  elif 'civitai.com' in url:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}
  else:
    print_message('잘못된 모델 URL입니다.')

def checkpoint_exists(): ### See if a checkpoint model exists in goodle drive ###
  files = os.listdir(output_path + '/checkpoints/') # get all files
  for file in files: # check .ckpt or .safetensors extension
    if file.endswith('.ckpt') or file.endswith('.safetensors'):
      return True
  return False # no checkpoint file

def download_models():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print_message('사용할 모델을 다운받고 있습니다.')
  # SD 1.5모델은 필수로 다운로드
  if not checkpoint_exists():
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
  if Deliberate_v2:
    downloadModel('https://civitai.com/api/download/models/15236')
  if DreamShaper_7:
    downloadModel('https://civitai.com/api/download/models/109123')
  if GhostMix_v20:
    downloadModel('https://civitai.com/api/download/models/76907')
  if RealisticVision_V30:
    downloadModel('https://civitai.com/api/download/models/105674')
  if MajicMIXRealistic_v6:
    downloadModel('https://civitai.com/api/download/models/94640')
  if AbyssOrangeMix3:
    downloadModel('https://civitai.com/api/download/models/17233')
  if CounterfeitV30_v30:
    downloadModel('https://civitai.com/api/download/models/57618')
  if ReVAnimated_v122:
    downloadModel('https://civitai.com/api/download/models/46846')
  if Lyriel_v16:
    downloadModel('https://civitai.com/api/download/models/72396')
  if NeverEndingDream_v122:
    downloadModel('https://civitai.com/api/download/models/64094')

  # if Model_URL:
  #     for m in Model_URL.split(','):
  #       %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  #       downloadModel(m)
  #       if Save_a_copy_in_Google_Drive and is_drive_used: # == GDriveSaveMode.ON:
  #         %cd {output_path}/checkpoints
  #         downloadModel(m)

  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def install_extensions():
  # extensions directory
  %cd {root}/stable-diffusion-webui/extensions

  if ControlNet:
    # %cd {root}/stable-diffusion-webui/extensions
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
#    %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
#    !pip install -r requirements.txt
    %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    %cd {root}/stable-diffusion-webui/extensions

  if Deforum:
    # %cd {root}/stable-diffusion-webui/extensions
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui
    #!cd {root}/stable-diffusion-webui/extensions/deforum; git checkout c42834645805e0f26172888b29f5a9210063db14

  if Regional_Prompter:
    # %cd {root}/stable-diffusion-webui/extensions
    !git clone https://github.com/hako-mikan/sd-webui-regional-prompter

  if Ultimate_SD_Upscale:
    # %cd {root}/stable-diffusion-webui/extensions
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111

  if Openpose_Editor:
    # %cd {root}/stable-diffusion-webui/extensions
    !git clone https://github.com/fkunn1326/openpose-editor

  if ADetailer:
    # %cd {root}/stable-diffusion-webui/extensions
    !git clone https://github.com/Bing-su/adetailer

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print_message('이미 python 3.10이 설치되어 있습니다.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
#    print("Create new ui-config.json file.")
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
#    print("Create new config.json file.")
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/config.json -O {output_path + '/config.json'}

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all checkpoints in the checkpoints folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  checkpoints_in_google_drive = glob.glob(output_path + '/checkpoints/*')
  print_message('구글 드라이브에 들어 있는 Checkpoint 모델: %s'%checkpoints_in_google_drive)
  for f in checkpoints_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print_message('구글 드라이브에 들어 있는 Upscaler 모델: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

def installExtensionsFromURL(urls):
  %cd {root}/stable-diffusion-webui/extensions
  for url in urls.split(','):
    !git clone {url}

updatePython()
!mkdir -p {root}
%cd {root}
# os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# A1111 first launch
%cd {root}/stable-diffusion-webui
#!git checkout -f 394ffa7b0a7fff3ec484bcd084e673a8b301ccc8 # 1.4.0
!git checkout -f f865d3e11647dfd6c7b2cdf90dde24680e58acd8 # 1.4.1
!COMMANDLINE_ARGS="--exit"  python launch.py

if is_drive_used: # == GDriveSaveMode.ON:
  initSaveGoogleDriveModelOnly()

download_models()
install_extensions()

# installExtensionsFromURL(Extensions_URL)

# clear()

%cd {root}/stable-diffusion-webui
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers'

if ngrok:
  args += f' --ngrok {ngrok} '
else:
  args += ' --share '
# args+= ' ' + Extra_args
#print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}